# Milestone 2 : The notebook #

## Introduction ## 

In this notebook the reader will find the pre-analysis of the dataset, some code sample to showcase that the ideas presented in the README are actually realisable and finally a couple of data visualisation to explore some ideas on how we will use the data at our disposal. 

In [64]:
import numpy as np
import pandas as pd
import folium
import urllib
import json
import socket
from ipwhois import IPWhois
import pycountry


To showcase what we can do with the current data we downloaded three examples files.

The way we take care of big data will be explained further in the notebook.

In [65]:
# Load data
# 20150218230000.export.CSV  20150218230000.gkg.csv  20150218230000.mentions.CSV
DATA_PATH = "data/"
MAP_PATH = DATA_PATH + "world.geo.json/countries/"
COUNTRY_CODE_DATA = DATA_PATH + "country-codes/data/country-codes.csv"

export_d = pd.read_csv(DATA_PATH + "20150218230000.export.CSV",sep='\t', names=get_export_names())
gkg = pd.read_csv(DATA_PATH + "20150218230000.gkg.csv",sep='\t', header=None)
mention_d = pd.read_csv(DATA_PATH + "20150218230000.mentions.CSV", sep="\t", names=get_mentions_names())

### Helpers functions ###

In the next cell there is a couple of function that helps us easily interact with the data.

```isNaN```: is a quick way to see when a field = float('nan').

```get_export_names```: Fetch the columns names for the export table.

```get_mentions_name```: Same as before but for the mentions table.

```get_map_site```: This function return two dictionnaries one that takes a website extension as an input and return the ```ISO3166-1-Alpha-3``` code for said country. The other do the same thing but the other way around. 

In [66]:
def isNaN(num):
    return num != num

def get_export_names():
    file = open(DATA_PATH + "event_table_name", "r")
    names = file.readlines()[0].split(" ")
    return names

def get_mentions_names():
    file = open(DATA_PATH + "mentions_table_name", "r")
    names = file.readlines()[0].split(" ")
    return names

def get_map_site():
    file = pd.read_csv(COUNTRY_CODE_DATA)
    return dict(zip(file['TLD'], file['ISO3166-1-Alpha-3'])), dict(zip(file['ISO3166-1-Alpha-3'], file['TLD']))


## Analysis of the data ##

### Peak at the data ###

In the next dataframe we will display the head of all three dataframe to get a concrete view at how it is inside.

In [67]:
export_d.head()

,GlobalEventID,Day,MounthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,410412347,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,4,"Waterkloof, Free State, South Africa",SF,SF03,77359,-30.3098,25.2971,-1299321,20150218230000,http://www.dailymaverick.co.za/article/2015-02...
1,410412348,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,4,"Bengaluru, Karnataka, India",IN,IN19,70159,12.9833,77.5833,-2090174,20150218230000,http://timesofindia.indiatimes.com/city/bengal...
2,410412349,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,4,"Great Southern, Victoria, Australia",AS,AS07,5387,-36.0667,146.4830,-1576477,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
3,410412350,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,1,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
4,410412351,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,2,"Idaho, United States",US,USID,NaN,44.2394,-114.5100,ID,20150218230000,http://www.eastidahonews.com/2015/02/neil-patr...


In [68]:
gkg.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,20150218230000-0,20150218230000,2,BBC Monitoring,as listed in Russian /BBC Monitoring/(c) BBC,"ARREST#400#political#4#Rossiya, Orenburgskaya ...","ARREST#400#political#4#Rossiya, Orenburgskaya ...",TERROR;REBELS;TAX_ETHNICITY;TAX_ETHNICITY_UKRA...,"TAX_FNCACT,2011;TAX_FNCACT,3449;TAX_FNCACT,381...","4#Budapest, Budapest, Hungary#RS#HU05#47.5#19....",...,"wc:693,c1.1:1,c1.2:1,c12.1:43,c12.10:58,c12.11...",NaN,NaN,NaN,NaN,1332|134||prisoners are treated humanely ; the...,"Channel One,628;Channel One,755;Channel One,14...","3,channels,476;3,channels reported,824;1,corre...",NaN,NaN
1,20150218230000-1,20150218230000,2,BBC Monitoring,"Al-Sharq al-Awsat website, London/BBC Monitori...",NaN,NaN,TAX_FNCACT;TAX_FNCACT_ENVOY;TAX_ETHNICITY;TAX_...,"IDEOLOGY,6790;IDEOLOGY,10592;KILL,1960;KILL,13...",1#Qatar#SA#QA#25.5#51.25#QA;1#Syria#SA#SY#35#3...,...,"wc:2376,c1.2:1,c1.4:7,c12.1:163,c12.10:276,c12...",NaN,NaN,NaN,NaN,332|26||greatly complicate matters#4494|37||ma...,"Emrullah Isler,83;Arab Spring,3571;Development...","2,fronts at the political,1455;2,main strategi...",NaN,NaN
2,20150218230000-2,20150218230000,1,wjon.com,http://wjon.com/wjon-news-on-demand-wednesday-...,NaN,NaN,MANMADE_DISASTER;MANMADE_DISASTER_WITHOUT_POWE...,"MANMADE_DISASTER,91;POWER_OUTAGE,91;",NaN,...,"wc:93,c12.1:7,c12.10:10,c12.12:5,c12.13:4,c12....",http://wac.450F.edgecastcdn.net/80450F/wjon.co...,http:/wac.450F.edgecastcdn.net/80450F/wjon.com...,NaN,https://youtube.com/channel/;https://youtube.c...,NaN,"Waite Park,125;City Food,233;Adobe Flash Playe...","500,people were without power,51;",NaN,NaN
3,20150218230000-3,20150218230000,1,wjol.com,http://www.wjol.com/common/more.php?m=15&r=3&i...,NaN,NaN,LEADER;TAX_FNCACT;TAX_FNCACT_GOVERNOR;TAX_POLI...,"TAX_FNCACT,131;TAX_POLITICAL_PARTY,664;TAX_FNC...","2#Wisconsin, United States#US#USWI#44.2563#-89...",...,"wc:103,c12.1:4,c12.10:13,c12.12:7,c12.13:6,c12...",NaN,NaN,NaN,NaN,NaN,"Governor Rauner,41;Speaker Michael Madigan,158...",NaN,NaN,NaN
4,20150218230000-4,20150218230000,1,straitstimes.com,http://www.straitstimes.com:80/news/world/unit...,NaN,NaN,DRONES;TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_MICE;...,"TAX_FNCACT,634;TAX_FNCACT,1248;TAX_FNCACT,907;...","1#United States#US#US#38#-97#US;3#Miami, Flori...",...,"wc:255,c12.1:10,c12.10:28,c12.12:11,c12.13:7,c...",http://www.straitstimes.com/sites/straitstimes...,NaN,NaN,NaN,NaN,"United States,583;Columbia University Medical ...","5,weeks,1194;",NaN,NaN


In [69]:
mention_d.head()

,GlobalEventId,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,ActorCharOffset,Actor2CharOffset,ActionCharOffset,InRawTest,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationinfo,Extras
0,410412347,20150218230000,20150218230000,1,dailymaverick.co.za,http://www.dailymaverick.co.za/article/2015-02...,19,-1,4594,4634,1,50,6665,-4.477612,NaN,NaN
1,410412348,20150218230000,20150218230000,1,indiatimes.com,http://timesofindia.indiatimes.com/city/bengal...,2,-1,300,344,1,50,2541,2.078522,NaN,NaN
2,410412349,20150218230000,20150218230000,1,voxy.co.nz,http://www.voxy.co.nz/entertainment/coast-new-...,4,-1,1297,1232,0,10,2576,7.517084,NaN,NaN
3,410412350,20150218230000,20150218230000,1,voxy.co.nz,http://www.voxy.co.nz/entertainment/coast-new-...,4,-1,1298,1233,1,20,2576,7.517084,NaN,NaN
4,410412351,20150218230000,20150218230000,1,eastidahonews.com,http://www.eastidahonews.com/2015/02/neil-patr...,1,-1,103,122,1,100,1432,0.000000,NaN,NaN


### A story of duplicates ###

Every element is present twice [...] and this part explain how we deal with it

### Columns we keep and why ###

On a side note the reader can find an exhaustive description of each column on the GDELT documentation (available in this repo in the PDF folder)

#### In export ####

- GlobalEventID : This column holds the unique ID for each event displayed in the dataset. It is also the link between the table export and the table mention.

- Actor1Geo_Lat, Actor1Geo_Long : These fields holds the geographical coordinates of the party included in the event. Its content will be display on a map.

- Actor2Geo_Lat, Actor2Geo_Long : Same thing as before. This holds informations about a second actor, if there is one, relative to a specifiv event.

- ActionGeo_Lat, ActionGeo_Long : Same thing as before but this time it holds the geographical coordinates of where the event was. Like the two last paragraph these information will be displayed on a world map.

- GoldsteinScale : This value which calculated how the stability of the country is after an event will be use to compute a homemade index named "Bias".

- SOURCEURL : Maybe we don't need to keep it ..?

#### In mentions ####  

- GlobalEventId : This fields holds the ID of the event from which the article speaks of. On the contrary of the export table an this fileds can be present multiple times through the database (one for each mention).

- MentionSourceName	: This fields holds the short version of the URL of the source. This will be used the geolocalise the soucre of the information since the information is not provided by GDELT

- Confidence : ???

- MentionDocTone : This fields holds a numerical value which quantified the hostility of an article. This feature is very useful in viewing the genral opinion of an article on an event wich will help us highlight biases in informations

In [70]:
# Maybe display the relevent data without all the garbage around  ?

## Visualisations ##

### Homemade tools for event visualisation ###

This section will display the functions that we created to do the analysis of the data each of them will have its own section for the reader to have a good understanding of how everything goes together.

#### treat_event: ####

This function display on map where an event heppened and if presents one or two actors.

##### Arguments: #####

- export : The GDELT export table

- mentions : The GDELT mentions table !!! Maybe not needed for this one ? !!!!

- f_map : The map on which the event should be display

##### How it works: #####

This function goes through two steps. 

First we look at the specific event in the export table. Once found we gather all interesting geographical informations.

Second, if present the data for each actor and the action is added to the map

In [71]:
def treat_event(export, mention, id_event, f_map):
    dat = export.loc[export['GlobalEventID'] == id_event]
    
    act_one_lat = dat['Actor1Geo_Lat'].values[0]
    act_one_long = dat['Actor1Geo_Long'].values[0]

    act_two_lat = dat['Actor2Geo_Lat'].values[0]
    act_two_long = dat['Actor2Geo_Long'].values[0]

    a_lat = dat['ActionGeo_Lat'].values[0]
    a_long = dat['ActionGeo_Long'].values[0]
    print(a_lat)
    print(a_long)
    src = dat['SOURCEURL\n'].values[0]
    
    if not isNaN(a_lat) and not isNaN(a_long):
        folium.Marker(
            location=[a_lat, a_long],
            popup=src,
            icon=folium.Icon(color="blue")
        ).add_to(f_map)
        
    if not isNaN(act_one_lat) and not isNaN(act_one_long):
        folium.Marker(
            location=[act_one_lat, act_one_long],
            icon=folium.Icon(color="green")
        ).add_to(f_map)
        
    if not isNaN(act_two_lat) and not isNaN(act_two_long):
        folium.Marker(
            location=[act_two_lat, act_two_long],
            icon=folium.Icon(color="red")
        ).add_to(f_map)

#### Example ####

In the next cell we took a random event and put it through the function. We can see that something happened in Australia because of New_Zeland.

It is possible to click on the action point to see the source of the article.

In [72]:
map_d = folium.Map(
    location=[-36, 146.5],
    zoom_start=4,
    tiles='Stamen Terrain'
)

treat_event(export_d, mention_d,410412349, map_d )

map_d


-36.0667
146.483


#### src_to_country ####

This function display on map a country with relation to the tone of its article.

##### Arguments : #####

- web : A website address
- f_map : A map on which we shall draw
- color : What colour should the country drawn be
- ip : How should we localise the article

##### How it works : ##### 

If ip = True : we locate the news outlet via its ip. First we do a DNS lookup to optain the IP of the website. From that we run a linux command ```whois``` which allows us to see where the website is based via a ```country``` field. From that we transform the alpha-2 country code into alpha-3 to fetch the json file containing the drawing information for said country.

If ip = False : we geolocalise the website using its extension. Earlier in the notebook we showcased in the helper function the creation of dictionnaries that offers a one to one mapping from website extensions to alpha-3 country code. Once the country code has been retrieved we use the same process as before to draw the country.

##### Note : #####

It is useful to highlight that the geolocalisation through DNS lookup and whois command needs an active internet connection

In [73]:
def src_to_country(web, f_map, color, ip = True): # Ture -> Country by IP, False -> Country by www.[..].country
    if ip:
        try:
            obj = IPWhois(socket.gethostbyname(web))
            results = obj.lookup_rdap(obj)
            country = pycountry.countries.get(alpha_2=results['asn_country_code'])
            layout = MAP_PATH + country.alpha_3 + ".geo.json"

            style_function = lambda x: {'fillColor': color}

            folium.GeoJson(layout, style_function).add_to(f_map)
        except:
            print("Country resolution for " + web + " has failed")
        
    else:
        site, _ = get_map_site() # get dictionnary
        country = site[str(".") + web.split('.')[-1]] # get web extension
        layout = MAP_PATH + country + ".geo.json"
        
        style_function = lambda x: {'fillColor': color}

        folium.GeoJson(layout, style_function).add_to(f_map)

#### tone_to_color ####

bla

##### Arguments : #####

p

##### How it works : #####

In [74]:
def tone_to_color(tone): # Tone is between -100 , 100 but -10 , 10 for 99% du time 
    if tone < 0:
        if tone < -5:
            return "red"
        else:
            return "pink"
    else:
        if tone > 5 :
            return "green"
        else:
            return "lightgreen"

#### treat_event ####

sa

##### Arguments : #####

p

##### How it works : #####

p

In [75]:
def treat_event(export, mention, id_event, f_map):
    dat = export.loc[export['GlobalEventID'] == id_event]
    display(dat)
    
    act_one_lat = dat['Actor1Geo_Lat'].values[0]
    act_one_long = dat['Actor1Geo_Long'].values[0]

    act_two_lat = dat['Actor2Geo_Lat'].values[0]
    act_two_long = dat['Actor2Geo_Long'].values[0]

    a_lat = dat['ActionGeo_Lat'].values[0]
    a_long = dat['ActionGeo_Long'].values[0]
    src = dat['SOURCEURL\n'].values[0]
    
    if isNaN(a_lat) or isNaN(a_long):
        pass
    else:
        folium.Marker(
            location=[a_lat, a_long],
            popup=src,
            icon=folium.Icon(color="blue")
        ).add_to(f_map)
        
    if isNaN(act_one_lat) or isNaN(act_one_long):
        pass
    else:
        folium.Marker(
            location=[act_one_lat, act_one_long],
            icon=folium.Icon(color="green")
        ).add_to(f_map)
        
    if isNaN(act_two_lat) or isNaN(act_two_long):
        pass
    else:
        folium.Marker(
            location=[act_two_lat, act_two_long],
            icon=folium.Icon(color="red")
        ).add_to(f_map)
        
    references = mention_d.loc[mention_d['GlobalEventId'] == id_event]
    display(references)
    
    for src, tone in zip(references['MentionSourceName'].values, references['MentionDocTone'].values):
        if src.split(".")[-1] == "com" or src.split(".")[-1] == "org":
            src_to_country(src, f_map, tone_to_color(tone), True)
        else:
            src_to_country(src, f_map, tone_to_color(tone), False)

In [76]:
events_id = export_d.loc[export_d['NumMentions'] == max(export_d['NumMentions'])]['GlobalEventID']

references = mention_d.loc[mention_d['GlobalEventId'] == events_id.values[0]]

references

,GlobalEventId,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,ActorCharOffset,Actor2CharOffset,ActionCharOffset,InRawTest,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationinfo,Extras
2147,410412879,20150218230000,20150218230000,1,yahoo.com,http://news.yahoo.com/islamic-state-militants-...,26,6847,6772,6854,1,80,7367,-4.187605,NaN,NaN
2148,410412879,20150218230000,20150218230000,1,kswo.com,http://www.kswo.com/story/28142565/islamic-sta...,27,7145,7070,7152,1,80,7596,-3.793382,NaN,NaN
2149,410412879,20150218230000,20150218230000,1,sfgate.com,http://www.sfgate.com/news/world/article/Islam...,32,10557,10482,10564,1,80,11165,-2.133633,NaN,NaN
2150,410412879,20150218230000,20150218230000,1,kxxv.com,http://www.kxxv.com/story/28142565/islamic-sta...,27,7141,7066,7148,1,80,7596,-3.787268,NaN,NaN
2151,410412879,20150218230000,20150218230000,1,ap.org,http://hosted2.ap.org/CAANR/0260ea4c3e85456b80...,31,10087,10012,10094,1,80,10716,-2.153667,NaN,NaN
2152,410412879,20150218230000,20150218230000,1,beaumontenterprise.com,http://www.beaumontenterprise.com/news/world/a...,26,7104,7029,7111,1,80,7603,-3.799515,NaN,NaN
2153,410412879,20150218230000,20150218230000,1,ap.org,http://hosted2.ap.org/APDEFAULT/cae69a7523db45...,31,10157,10082,10164,1,80,10824,-2.136259,NaN,NaN
2154,410412879,20150218230000,20150218230000,1,yahoo.com,https://uk.news.yahoo.com/islamic-state-milita...,26,7104,7029,7111,1,80,7601,-3.799515,NaN,NaN
2155,410412879,20150218230000,20150218230000,1,washingtonpost.com,http://www.washingtonpost.com/world/middle_eas...,28,7609,7534,7616,1,80,8183,-3.484848,NaN,NaN
2156,410412879,20150218230000,20150218230000,1,therepublic.com,http://www.therepublic.com/view/story/5563550e...,27,7595,7520,7602,1,80,8068,-3.435115,NaN,NaN


In [77]:
print (socket.gethostbyname('postzambia.com')) 

192.64.119.171


In [78]:
#src_to_country('dailymail.co.uk', m, "ROT", False) # ENGLAND
src_to_country('dailytimes.com.pk', m, tone_to_color(-3), False) # No color encoding rn # PAKISTAN 
src_to_country('kswo.com', m, tone_to_color(7)) # cast by ip # USa

/usr/lib/python3.7/site-packages/ipwhois/net.py:138: UserWarning: allow_permutations has been deprecated and will be removed. It is no longer needed, due to the deprecation of asn_alts, and the addition of the asn_methods argument.
  warn('allow_permutations has been deprecated and will be removed. '
/usr/lib/python3.7/site-packages/ipwhois/asn.py:178: UserWarning: IPASN._parse_fields_dns() has been deprecated and will be removed. You should now use IPASN.parse_fields_dns().
  warn('IPASN._parse_fields_dns() has been deprecated and will be '


In [79]:

    

map_d = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=2,
    tiles='Stamen Terrain'
)

treat_event(export_d, mention_d,410412879, map_d)

map_d


,GlobalEventID,Day,MounthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
532,410412879,20150218,201502,2015,2015.1315,COP,POLICE,NaN,NaN,NaN,...,4,"Sirte, Sha`biyat Surt, Libya",LY,LY76,19099,31.2089,16.5887,-3178329,20150218230000,http://hosted2.ap.org/CAANR/0260ea4c3e85456b80...


,GlobalEventId,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,ActorCharOffset,Actor2CharOffset,ActionCharOffset,InRawTest,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationinfo,Extras
2147,410412879,20150218230000,20150218230000,1,yahoo.com,http://news.yahoo.com/islamic-state-militants-...,26,6847,6772,6854,1,80,7367,-4.187605,NaN,NaN
2148,410412879,20150218230000,20150218230000,1,kswo.com,http://www.kswo.com/story/28142565/islamic-sta...,27,7145,7070,7152,1,80,7596,-3.793382,NaN,NaN
2149,410412879,20150218230000,20150218230000,1,sfgate.com,http://www.sfgate.com/news/world/article/Islam...,32,10557,10482,10564,1,80,11165,-2.133633,NaN,NaN
2150,410412879,20150218230000,20150218230000,1,kxxv.com,http://www.kxxv.com/story/28142565/islamic-sta...,27,7141,7066,7148,1,80,7596,-3.787268,NaN,NaN
2151,410412879,20150218230000,20150218230000,1,ap.org,http://hosted2.ap.org/CAANR/0260ea4c3e85456b80...,31,10087,10012,10094,1,80,10716,-2.153667,NaN,NaN
2152,410412879,20150218230000,20150218230000,1,beaumontenterprise.com,http://www.beaumontenterprise.com/news/world/a...,26,7104,7029,7111,1,80,7603,-3.799515,NaN,NaN
2153,410412879,20150218230000,20150218230000,1,ap.org,http://hosted2.ap.org/APDEFAULT/cae69a7523db45...,31,10157,10082,10164,1,80,10824,-2.136259,NaN,NaN
2154,410412879,20150218230000,20150218230000,1,yahoo.com,https://uk.news.yahoo.com/islamic-state-milita...,26,7104,7029,7111,1,80,7601,-3.799515,NaN,NaN
2155,410412879,20150218230000,20150218230000,1,washingtonpost.com,http://www.washingtonpost.com/world/middle_eas...,28,7609,7534,7616,1,80,8183,-3.484848,NaN,NaN
2156,410412879,20150218230000,20150218230000,1,therepublic.com,http://www.therepublic.com/view/story/5563550e...,27,7595,7520,7602,1,80,8068,-3.435115,NaN,NaN


/usr/lib/python3.7/site-packages/ipwhois/net.py:138: UserWarning: allow_permutations has been deprecated and will be removed. It is no longer needed, due to the deprecation of asn_alts, and the addition of the asn_methods argument.
  warn('allow_permutations has been deprecated and will be removed. '
/usr/lib/python3.7/site-packages/ipwhois/asn.py:178: UserWarning: IPASN._parse_fields_dns() has been deprecated and will be removed. You should now use IPASN.parse_fields_dns().
  warn('IPASN._parse_fields_dns() has been deprecated and will be '


Country resolution for beaumontenterprise.com has failed
